## Vulnerability in the US at Census Tract Level

In [ ]:
import geopandas as gpd
import pandas as pd
import pathlib

import matplotlib.pyplot as plt

from cartopy import crs as ccrs

import os
import sys
import subprocess
import datetime
import platform
import datetime

In [ ]:
albo = ccrs.AlbersEqualArea(
    central_longitude=-96,
    central_latitude=37.5,
    false_easting=0.0,
    false_northing=0.0,
    standard_parallels=(29.5, 45.5),
    globe=None,
)

In [ ]:
import glob
path=r'census_shp'

file=glob.glob(os.path.join(path, 'cb_2018_**_tract_500k.shp'))
# print(file)
print(len(file))

In [ ]:
## Import the census tracts

basemap= gpd.GeoDataFrame()
for f in file:
    basemap = pd.concat([basemap, gpd.read_file(f)], axis=0)
    
basemap = basemap[['GEOID','AFFGEOID','STATEFP','geometry']]
basemap.head()

In [ ]:
basemap.crs

In [ ]:
## Change the projection
basemap = basemap.to_crs(albo.proj4_init)

In [ ]:
## Change the position of Alaska and some islands

m = basemap.STATEFP =='15'
basemap[m] = basemap[m].set_geometry(basemap[m].translate(5000000,-1500000))

n = basemap.STATEFP == '02'
basemap[n] = basemap[n].set_geometry(basemap[n].scale(.5,.5,.5,origin=(-3000000, 4000000)).translate(1000000, -4800000))

State Boundary used in Maps

In [ ]:
## Import the state boundaries
state_boundary = gpd.read_file('state_shp/cb_2018_us_state_500k.shp')
state_boundary.plot()

In [ ]:
state_boundary = state_boundary.to_crs(albo.proj4_init)

m = state_boundary.STATEFP =='15'
state_boundary[m] = state_boundary[m].set_geometry(state_boundary[m].translate(5000000,-1500000))

n = state_boundary.STATEFP == '02'
state_boundary[n] = state_boundary[n].set_geometry(state_boundary[n].scale(.5,.5,.5,origin=(-3000000, 4000000)).translate(1000000, -4800000))

# Exposure

In [ ]:
## Import data

exposure = pd.read_csv('census_data/exposure_census.csv', dtype = {'State':str,"GEOID":str,"GEO_ID":str})
exposure.head()

In [ ]:
exposure_map = pd.merge(exposure,basemap,on='GEOID',how='right')
exposure_map = gpd.GeoDataFrame(exposure_map,crs = albo.proj4_init)
exposure_map.crs

In [ ]:
exposure_map.info()

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
# plt.rc('font',family='Times New Roman')

ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

exposure_map.plot(
    column='min_max_ex',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=5,
    legend_kwds={'loc': 'upper left',
                 'title': 'Exposure',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # Title of legend
                 'fontsize': 11, # Font of legend
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Exposure', fontdict={'family':'Times New Roman','weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Exposure_USA.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

exposure_map.plot(
    column='fuel',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Fuel Consumption (gallons)',
                 "fmt": "{:.2f}",
                 'title_fontsize': 10, 
                 'fontsize': 10, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Fuel Consumption in the US', fontdict={'weight':'bold','size': 20}, pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Fuel Consumption_USA.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

exposure_map.plot(
    column='Price',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
    k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Fuel Price ($/gallon)',
                 "fmt": "{:.3f}",
                 'title_fontsize': 12, 
                 'fontsize': 12, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Fuel Price in the US', fontdict={'weight':'bold','size': 20}, pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Price_USA.png', facecolor='w', dpi=500)

# Sensitivity

In [ ]:
sensitivity = pd.read_csv("census_data/sensitivity_census.csv", dtype={"Geocode": str})

In [ ]:
sensitivity.head()

In [ ]:
sensitivity_map = pd.merge(sensitivity,basemap,left_on='Geocode',right_on='GEOID',how='right').drop(['Geocode'],axis=1)

In [ ]:
sensitivity_map = gpd.GeoDataFrame(sensitivity_map,crs = albo.proj4_init)

In [ ]:
sensitivity_map.crs

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

sensitivity_map.plot(
    column='min_max_sens',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Sensitivity',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Sensitivity', fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Sensitivity_USA.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

sensitivity_map.plot(
    column='X7',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'X7',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Sensitivity', fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/X7.png', facecolor='w', dpi=500)

# Adaptive Capacity

In [ ]:
adaptive_capacity = pd.read_csv('census_data/adaptive_capacity_census.csv',dtype={"GEO_ID":str})
adaptive_capacity.head()

In [ ]:
adaptive_capacity_map = pd.merge(adaptive_capacity,basemap,left_on='GEO_ID',right_on='GEOID',how='right').drop(['GEO_ID'],axis=1)
adaptive_capacity_map.info()

In [ ]:
adaptive_capacity_map = gpd.GeoDataFrame(adaptive_capacity_map,crs = albo.proj4_init)
adaptive_capacity_map.crs

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

adaptive_capacity_map.plot(
    column='min_max_ac',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Adaptive Capacity',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Adaptive Capacity',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Adaptive Capacity_USA.png', facecolor='w', dpi=500)

# Vulnerabilty Score

In [ ]:
e = exposure_map[['GEOID','min_max_ex']]
s = sensitivity_map[['GEOID','min_max_sens']]
ac = adaptive_capacity_map[['GEOID','min_max_ac']]

In [ ]:
vulnerablity = pd.merge(e, s, on="GEOID")
vulnerablity = pd.merge(vulnerablity, ac, on="GEOID")
vulnerablity.head()

In [ ]:
vulnerablity.info()

In [ ]:
vulnerablity.fillna(0, inplace=True)

In [ ]:
vulnerablity['vul_score_mulpi'] = vulnerablity.apply(lambda x: x["min_max_ex"]*(x["min_max_sens"]-x["min_max_ac"]), axis=1)

In [ ]:
vulnerablity['vul_score_add'] = vulnerablity.apply(lambda x: x["min_max_ex"]+x["min_max_sens"]-x["min_max_ac"], axis=1)

In [ ]:
vulnerablity.head()

In [ ]:
vulnerablity.to_csv('Vulnerability Score.csv')

In [ ]:
vulnerablity_map = pd.merge(vulnerablity,basemap,on='GEOID')
vulnerablity_map = gpd.GeoDataFrame(vulnerablity_map,crs = albo.proj4_init)

In [ ]:
vulnerablity_map.crs

In [ ]:
vulnerablity_map_pos = vulnerablity_map[vulnerablity_map['vul_score_mulpi']>=0]
vulnerablity_map_neg = vulnerablity_map[vulnerablity_map['vul_score_mulpi']<0]
print(vulnerablity_map_pos['vul_score_mulpi'].mean(),vulnerablity_map_pos['vul_score_mulpi'].std())
print(vulnerablity_map_neg['vul_score_mulpi'].mean(),vulnerablity_map_neg['vul_score_mulpi'].std())

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

vulnerablity_map.plot(
    column='vul_score_add',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Vul_score_add.png', facecolor='w', dpi=500)

## EV Adoption in IL

In [ ]:
vul_IL = pd.read_excel('IL CV-EV/vul_score CV-EV.xlsx',dtype={'CountyFIPs':str})
vul_IL.info()

In [ ]:
IL_map = gpd.read_file('IL_county_shp/IL_BNDY_County_Py.shp',dtype={'CO_FIPS':str})
IL_map

In [ ]:
IL_map_COOK = IL_map[IL_map['COUNTY_NAM']=='COOK']
IL_map_Alexander = IL_map[IL_map['COUNTY_NAM']=='ALEXANDER']

In [ ]:
IL_vul_map = pd.merge(vul_IL, IL_map, left_on = 'County', right_on = 'COUNTY_NAM')

In [ ]:
IL_vul_map = gpd.GeoDataFrame(IL_vul_map,crs = 4269)

In [ ]:
IL_vul_map['2020CV_new'] = IL_vul_map.apply(lambda x: abs(x['2020CV']), axis=1)

In [ ]:
IL_vul_map['2020EV_new'] = IL_vul_map.apply(lambda x: abs(x['2020EV']), axis=1)

In [ ]:
IL_vul_map['dif_20'] = IL_vul_map.apply(lambda x: x['2020CV_new'] - x['2020EV_new'], axis=1)

In [ ]:
print(IL_vul_map['2020CV_new'].max(),IL_vul_map['2020CV_new'].min())

In [ ]:
print(IL_vul_map['2020EV_new'].max(),IL_vul_map['2020EV_new'].min())

In [ ]:
IL_vul_map['dif_50'] = IL_vul_map.apply(lambda x: x['2050CV'] - x['2050EV'], axis=1)  

In [ ]:
IL_vul_map['2050CV_new'] = IL_vul_map.apply(lambda x: abs(x['2050CV']), axis=1)

In [ ]:
IL_vul_map['2050EV_new'] = IL_vul_map.apply(lambda x: abs(x['2050EV']), axis=1)

In [ ]:
print(IL_vul_map['2050CV_new'].max(),IL_vul_map['2050CV_new'].min())

In [ ]:
print(IL_vul_map['2050EV_new'].max(),IL_vul_map['2050EV_new'].min())

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10),dpi=300)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

IL_vul_map.plot(
    column='2050CV_new',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
#    scheme='NaturalBreaks',
#     k=7,
#     legend_kwds={'loc': 'lower left',
#                  'title': 'Vulnerability Score',
#                  "fmt": "{:.3f}",
#                  'title_fontsize': 14, 
#                  'fontsize': 11, 
#                  'shadow': True
#                 },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2,
    vmin = 0,
    vmax = 0.7442712535493357
)
IL_map.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)
IL_map_COOK.boundary.plot(ax=ax,edgecolor='red',linewidth=1)
IL_map_Alexander.boundary.plot(ax=ax,edgecolor='red',linewidth=1)

# for index, row in IL_map_COOK.iterrows():
#     xy = row['geometry'].centroid.coords[:]
#     xytext = row['geometry'].centroid.coords[:]
#     name = row['COUNTY_NAM']+str(' COUNTY')
#     ax.annotate(name, xy=xy[0], xytext=xytext[0],fontsize=12, color='red',
#                  horizontalalignment='center', verticalalignment='center')

# for index, row in IL_map_Alexander.iterrows():
#     xy = row['geometry'].centroid.coords[:]
#     xytext = row['geometry'].centroid.coords[:]
#     name = row['COUNTY_NAM']+str(' COUNTY')
#     ax.annotate(name, xy=xy[0], xytext=xytext[0],fontsize=12, color='red',
#                  horizontalalignment='center', verticalalignment='center')

# xy_COOK = IL_map_COOK['geometry'].centroid.coords[:]
# xytext_COOK = IL_map_COOK['geometry'].centroid.coords[:]
# ax.annotate(IL_map_COOK['COUNTY_NAM'], xy=xy_COOK[0], xytext=xytext_COOK[0],horizontalalignment='center', verticalalignment='center')


ax.axis('on')
#ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/IL_2050CV.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10),dpi=300)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

IL_vul_map.plot(
    column='2050EV_new',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
#    scheme='NaturalBreaks',
#     k=7,
#     legend_kwds={'loc': 'lower left',
#                  'title': 'Vulnerability Score',
#                  "fmt": "{:.3f}",
#                  'title_fontsize': 14, 
#                  'fontsize': 11, 
#                  'shadow': True
#                 },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2,
    vmin = 0,
    vmax = 0.7442712535493357
)
IL_map.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)
IL_map_COOK.boundary.plot(ax=ax,edgecolor='red',linewidth=1)
IL_map_Alexander.boundary.plot(ax=ax,edgecolor='red',linewidth=1)

ax.axis('on')
#ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/IL_2050EV.png', facecolor='w', dpi=500)